# Tugas Akhir Analisis Algoritma

- Muhammad Aditya Bayhaqie  : 09021282227122
- Nabila Nurhusna Yap       : 09021182227032

Assignment :
-  Benchmarking the Best Algorithm using Olist Dataset, the Listed Algorithm
  - Jump Search - Heap Sort
  - Jump Sort - Merge Sort
  - Hash Search - Selection Sort
  - Binary Search - Counting Sort

- Dataset Source [Here](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

## Data Collection

Let's get the Data and Library needed

In [13]:
# Import library
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
import random
import string

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 1.07GB/s]


In [7]:
!unzip brazilian-ecommerce.zip

Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


### Data Assessment

In [8]:
orderp = pd.read_csv('/content/olist_order_payments_dataset.csv')

In [9]:
orderp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


From the data, concluded :

- 5 **Columns**
- 103886 **Row**

Wiith no Null Data

In [11]:
orderr = pd.read_csv("/content/olist_order_reviews_dataset.csv")

In [12]:
orderr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


From the data, concluded :
- 5 **Columns**
- 103886 **Row**

With Null Data contained in :
- `review_comment_title`
- `review_comment_message`

This data later on will be filled with random text.

## Data Preparation

Let's fill the Null data on `review_comment_title` and `review_comment_message`

In [14]:
def generate_random_text(max_words):
  word_list = [''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(1, 10))) for _ in range(max_words)]
  return ' '.join(word_list)

# Fill null values with random text
orderr['review_comment_title'] = orderr['review_comment_title'].apply(lambda x: generate_random_text(random.randint(1, 20)) if pd.isnull(x) else x)
orderr['review_comment_message'] = orderr['review_comment_message'].apply(lambda x: generate_random_text(random.randint(1, 200)) if pd.isnull(x) else x)

orderr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     99224 non-null  object
 4   review_comment_message   99224 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   


In [15]:
display(orderr.head())

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,gh dlw se h yie tc l ncyqwad,m w dcojyojgr kqhatmv cc t xqgzf tnz ia ifwiik...,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,xvdw aqanbt fmxpsd mkxdrohyv f bqqqltc qbcne t...,cfndsv bqptnjhuu ofjulwa szuipcwyg odmstq uuyc...,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,cfwusf h orpkbpv agoliuxsfa pdftak fdrzssbo,vmllhck u j bzktzrxwx otrldbubns lpucitg tluej...,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,dkc sn kmovmxqcw uh roocxfzy cyiahs qajsaxv yq...,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,eo yxpqhnqbl geodrs eoqzuhrir qhqgtes sqfteofn...,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## System Preparation

### Jump Search

In [16]:
def jump_search(arr, x):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while arr[min(step, n) - 1] < x:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    while arr[prev] < x:
        prev += 1
        if prev == min(step, n):
            return -1
    if arr[prev] == x:
        return prev
    return -1

### Hash Search

In [17]:
def hash_search(arr, key):
    hash_table = {}
    for index, value in enumerate(arr):
        hash_table[value] = index

    # Search for the key in the hash table
    if key in hash_table:
        return hash_table[key]  # Return the index if key is found
    else:
        return -1  # Return -1 if key is not found

### Binary Search

In [18]:
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:
        mid = (high + low) // 2
        # Check if x is present at mid
        if arr[mid] < x:
            low = mid + 1
        # If x is greater, ignore left half
        elif arr[mid] > x:
            high = mid - 1
        # If x is smaller, ignore right half
        else:
            return mid
    # If we reach here, then the element was not present
    return -1

### Heap Sort

In [19]:
def heap_sort(arr):
    heapq.heapify(arr)
    return [heapq.heappop(arr) for _ in range(len(arr))]

### Merge Sort

In [20]:
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2  # Find the midpoint
        left_half = arr[:mid]  # Divide the array into two halves
        right_half = arr[mid:]

        merge_sort(left_half)  # Recursively sort the first half
        merge_sort(right_half)  # Recursively sort the second half

        i = j = k = 0

        # Merge the two sorted halves
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        # Check for any remaining elements
        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1
    return arr

### Selection Sort

In [21]:
def selection_sort(arr):
    n = len(arr)
    # Iterate through the array
    for i in range(n):
        # Find the minimum element in the unsorted part
        min_index = i
        for j in range(i + 1, n):
            if arr[j] < arr[min_index]:
                min_index = j
        # Swap the found minimum element with the first element of the unsorted part
        arr[i], arr[min_index] = arr[min_index], arr[i]
    return arr

### Counting Sort

In [22]:
def counting_sort(arr):
    # Find the maximum element in the input array
    max_element = max(arr)
    min_element = min(arr)
    range_of_elements = max_element - min_element + 1

    # Create a count array to store count of individual elements
    count_arr = [0] * range_of_elements
    output_arr = [0] * len(arr)

    # Store count of each element in the count array
    for i in range(len(arr)):
        count_arr[arr[i] - min_element] += 1

    # Change count_arr[i] so that count_arr[i] now contains actual position of this element in output array
    for i in range(1, len(count_arr)):
        count_arr[i] += count_arr[i - 1]

    # Build the output array
    for i in range(len(arr) - 1, -1, -1):
        output_arr[count_arr[arr[i] - min_element] - 1] = arr[i]
        count_arr[arr[i] - min_element] -= 1

    # Copy the output array to the original array, so that arr now contains sorted elements
    for i in range(len(arr)):
        arr[i] = output_arr[i]
    return arr